In [ ]:
# https://www.youtube.com/watch?v=DqtlR0y0suo

In [2]:
#Sending GET request through Insomnia API to nfl/odds page. Data does not contain props. Will have to run this script 2x
import requests
import pandas as pd


get_content_url= "https://content.unabated.com/markets/game-odds/v_gameodds.json?"
get_books_url = "https://api-k.unabated.com/api/users/settings"


#Main Lines URL
Privilege_content = "https://content.unabated.com/markets/game-odds/b_gameodds.json"
#This one is when I am logged in

In [3]:
url = Privilege_content
#returns isPrivileged: true for premium sportsbooks

payload = ""
headers = {
    "authority": "content.unabated.com",
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "dnt": "1",
    "origin": "https://unabated.com",
    "referer": "https://unabated.com/",
    "sec-ch-ua": "^\^Google",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "^\^Windows^^",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "x-amz-cf-v-id": "dNSob7rQOiJFE_59NDSm7tuZ6ed5MA_8w8CHVqr8tGSFYpgzEUqBuQ=="
}
#We can customize the headers we include ^ I left as it came
r = requests.request("GET", url, data=payload, headers=headers)

# print(r.json())
# all good

data = r.json()

marketSources = (data["marketSources"])
teams = (data["teams"])
gameOddsEvents = (data["gameOddsEvents"])


df_sportsbooks = pd.DataFrame(marketSources)
df_teams = pd.DataFrame(teams)
df_teams=df_teams.transpose()


Transforming the SportsBooks Reference Table

In [4]:
#Removing columns for building purposes 
#Note a bunch of other sportsbooks exist but are not shown on page. 

df_sb = pd.DataFrame().assign(Name=df_sportsbooks['name'],bookId=df_sportsbooks['id'],isActive=df_sportsbooks['isActive'],
isPrivileged=df_sportsbooks['isPrivileged'],statusId=df_sportsbooks['statusId'],modifiedDate=df_sportsbooks['modifiedOn'])


#Creating Binary flags 
 
df_sb['nyBook']= 0
df_sb['sharpBook']= 0


In [5]:
df_nyBooks= pd.DataFrame(
    {"Name" : ["Caesars", "DraftKings", "FanDuel", "FanDuel - Delayed", "BetMGM", "PointsBet", "BetRivers", "WynnBet", "Resorts", "BallyBet"],
    'nyBook': [1,1,1,1,1,1,1,1,1,1]
    }
)
df_sharpBooks = pd.DataFrame(
    {"Name" :['Unabated','Pinnacle','Pinnacle - Delayed','Bookmaker','Prophet Exchange'],
    'sharpBook': [1,1,1,1,1]
    }
)

In [6]:
df_sb=df_sb.set_index('Name')
df2 = df_nyBooks.set_index('Name')
df_sb.update(df2)

df3=df_sharpBooks.set_index('Name')
df_sb.update(df3)

In [7]:
df_sb

#This is the Sportsbook Reference table. Might also want to change the number format for the dates

,bookId,isActive,isPrivileged,statusId,modifiedDate,nyBook,sharpBook
Name,,,,,,,
Unabated,49,True,True,0,2022-03-16T21:38:37.525262,0.0,1.0
Pinnacle - Delayed,58,True,False,2,2022-08-17T12:18:20.442277,0.0,1.0
theScore,60,True,False,1,2022-08-25T18:02:36.780451,0.0,0.0
FanDuel - Delayed,57,True,False,2,2022-06-09T14:45:56.826297,1.0,0.0
DraftKings,1,True,False,1,2022-06-28T15:42:41.593641,1.0,0.0
...,...,...,...,...,...,...,...
FoxBet,21,False,False,2,2022-12-13T18:38:54.309456,0.0,0.0
Circa,6,True,True,2,2022-12-14T13:42:03.734851,0.0,0.0
BetRivers,17,False,False,2,2022-12-17T19:21:04.815139,1.0,0.0


In [34]:
df_sb.to_csv('sportsbooksTable.csv')

________________________________________________________________________________________


Now we create the Teams table. 

The eventId is the unique gameID which two teams will share. Not positive on what sideId is yet, but it distinguishes one team from another inside of a cell. 


In [8]:
#slimming down

df_teams_refined = pd.DataFrame().assign(teamId=df_teams['id'],teamName=df_teams['name'],abbreviation=df_teams['abbreviation'],
eventId=df_teams['eventId'],sideId=df_teams['sideId'],leagueId=df_teams['leagueId'],divisionId=df_teams['divisionId'],
modifiedDate=df_teams['modifiedOn'])

df_teams_refined
#This is our Teams Table

,teamId,teamName,abbreviation,eventId,sideId,leagueId,divisionId,modifiedDate
20,20,New Orleans Saints,NO,21915,1,1,8,0001-01-01T00:00:00
2,2,Atlanta Falcons,ATL,21915,0,1,8,0001-01-01T00:00:00
32,32,Washington Commanders,WAS,21916,1,1,5,0001-01-01T00:00:00
21,21,New York Giants,NYG,21916,0,1,5,0001-01-01T00:00:00
6,6,Chicago Bears,CHI,21917,1,1,7,0001-01-01T00:00:00
...,...,...,...,...,...,...,...,...
662,662,North Carolina,NC,39832,1,2,9,0001-01-01T00:00:00
673,673,Oregon,ORE,39832,0,2,9,0001-01-01T00:00:00
1143,1143,Eagles Trinity Baptist,TBC,39899,0,4,None,0001-01-01T00:00:00
1170,1170,Montana State,MOS,39966,0,2,10,0001-01-01T00:00:00


In [33]:
df_teams_refined.to_csv('df_teams_ref_out.csv')



__________________________________________________________________________________

Now we tackle the GameOdd Table

In [10]:
data = []
for i in gameOddsEvents:
  for j in gameOddsEvents[i]:
    event_teams = j['eventTeams']
    id_0 = event_teams['0']['id']
    id_1 = event_teams['1']['id']
    
    # Extract the americanPrice values from gameOddsMarketSourcesLines
    game_odds_market_sources_lines = j['gameOddsMarketSourcesLines']
    for market_source, market_data in game_odds_market_sources_lines.items():
      siX= market_source
      for bet_type, bet_data in market_data.items():
            american_price = bet_data['americanPrice']
            marketLineId= bet_data['marketLineId']
            marketId= bet_data['marketId']
            marketSourceId= bet_data['marketSourceId']
            points= bet_data['points']
            price= bet_data['price']
            sourcePrice= bet_data['sourcePrice']
            btX = bet_type
            # Store the extracted data in a tuple
            row = (i, j['eventId'],j['eventStart'],j['eventEnd'] ,id_0, id_1, siX, btX, marketLineId, marketSourceId,
            marketId, points,american_price, price, sourcePrice)
            data.append(row)



# Create the DataFrame with the extracted data
df_MainLineOdds = pd.DataFrame(data, columns=['lgX', 'eventId','eventStart','eventEnd','Teamid_0', 'Teamid_1', 'siX', 'btX', 'marketLineId','marketSourceId','marketId','points','american_price','price','sourcePrice'])
df_MainLineOdds['lgX_league']= df_MainLineOdds['lgX'].str.get(2)
df_MainLineOdds['lgX_pt']= df_MainLineOdds['lgX'].str.get(6)
df_MainLineOdds['lgX_stage']= df_MainLineOdds['lgX'].str[8:]
df_MainLineOdds['siX_side']= df_MainLineOdds['siX'].str.get(2)

df_MainLineOdds

,lgX,eventId,eventStart,eventEnd,Teamid_0,Teamid_1,siX,btX,marketLineId,marketSourceId,marketId,points,american_price,price,sourcePrice,lgX_league,lgX_pt,lgX_stage,siX_side
0,lg1:pt1:pregame,21915,2022-12-18T18:00:00,None,2,20,si1:ms1:an0,bt2,67201099,1,474009,-4.0,-115,-115,1.869565,1,1,pregame,1
1,lg1:pt1:pregame,21915,2022-12-18T18:00:00,None,2,20,si1:ms1:an0,bt1,67201109,1,667390,0.0,-200,-200,1.500000,1,1,pregame,1
2,lg1:pt1:pregame,21915,2022-12-18T18:00:00,None,2,20,si1:ms1:an0,bt3,211899092,1,1447795,43.5,-110,-110,1.909091,1,1,pregame,1
3,lg1:pt1:pregame,21915,2022-12-18T18:00:00,None,2,20,si1:ms2:an0,bt2,211906005,2,474009,-4.5,-105,-105,-105.000000,1,1,pregame,1
4,lg1:pt1:pregame,21915,2022-12-18T18:00:00,None,2,20,si1:ms2:an0,bt1,211906001,2,667390,0.0,-200,-200,-200.000000,1,1,pregame,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48005,lg1:pt3:live,39801,2022-12-17T21:30:00,None,3,8,si1:ms54:an0,bt3,224610128,54,1492516,18.5,100,100,100.000000,1,3,live,1
48006,lg1:pt3:live,39801,2022-12-17T21:30:00,None,3,8,si1:ms59:an0,bt2,224610979,59,1492514,0.0,-120,-120,-120.000000,1,3,live,1
48007,lg1:pt3:live,39801,2022-12-17T21:30:00,None,3,8,si1:ms59:an0,bt3,224611216,59,1492516,19.0,-105,-105,-105.000000,1,3,live,1
48008,lg1:pt3:live,39801,2022-12-17T21:30:00,None,3,8,si1:ms53:an0,bt2,224609146,53,1492514,0.0,-115,-115,-115.000000,1,3,live,1


In [32]:
df_MainLineOdds.to_csv('MainLineOdds.csv')

Write to MainLinesMaster

In [11]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(r'C:\Users\ammoo\OneDrive\Desktop\PythonOutData\Main Lines Master.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df_MainLineOdds.to_excel(writer, sheet_name='MainLineOdds')
df_teams_refined.to_excel(writer, sheet_name='Teams')
df_sb.to_excel(writer, sheet_name='Books')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

c:\Users\ammoo\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
